In [1]:
import json
import requests

In [2]:
url = "http://localhost:11434/api/chat"

In [3]:
model = "llama3.1:8b" # [llama3.1:8b] [gemma2:9b]

In [4]:
system_prompt = '''
IDDENTITY AND PURPOSE
--------------------
You are an EXPERT in text PSEUDONYMIZATION.
Your task is to ONLY DETECT defined entites and PRODUCE type-compliant PSEUDONYMs.
You will be provided some SAMPLE INPUT and corresponding SAMPLE_OUTPUT to be used as examples and guide for you.
You will also be provided the list of total 14 different ENTITY TYPES AND THEIR DEFINITIONS to be used as knowledge.
You will ONLY output in a format similar to SAMPLE OUTPUT format, no ADDITIONAL text or EXPLANATIONS.


ENTITY TYPES AND THEIR DEFINITIONS
----------------------------------
1. CITY = stands for villages, towns, cities, metropolitan areas and regions smaller than a state
2. DATE = covers all sorts of date descriptions
3. EMAIL = covers all types of email addresses in the texts
4. FAMILY = covers all family names
5. FEMALE = female given names, includes nicknames and initials
6. MALE = male given names, includes nicknames and initials
7. ORG = includes all types of legal actors such as companies, brands, institutions and agencies, etc.
8. PHONE = includes phone numbers and fax numbers
9. STREET = includes all kinds of street names
10. STREETNO = street numbers that appear in location details
11. UFID = to capture persons (students, customers, employees, members of social security systems, authors, etc.)
12. URL = includes other forms of domain names
13. USER = covers all kinds of invented usernames for IT systems and platforms
14. ZIP = zip codes in location details



SAMPLE INPUT: 1
---------------
Besonders nicht bei Italo!


-- 
Zitat:
ACHTUNG "SuPer Plexer Trottel"
Niedernbergweg 5, 91160 Jeggen, 130/3177345
http://onn.mmewxds.sps/lucy-o93112.jeaj

SAMPLE OUTPUT: 1
----------------
MALE: Italo **Fernando**; STREET: Niedernbergweg **Blütenring**; STREETNO: 5 **7**; ZIP: 91160 **88521**; CITY: Jeggen **Nonnenberg**; PHONE: 130/3177345 **664/8651272**; URL: http://onn.mmewxds.sps/lucy-o93112.jeaj **http://leb.uizotxi.kba/dabw-w08293.apqp**


SAMPLE INPUT: 2
---------------
genau, das ist kurz nach dem Kamelmarkt.


lol


-- 
Zitat:
ACHTUNG "Turbotrottel"
Exerzierplatzstraße 5, 91386 Oberbaumgarten, 312/4603663
http://oqq.yyzlnom.zbm/hiog-z30270.ubgo

SAMPLE OUTPUT: 2
----------------
STREET: Exerzierplatzstraße **Töpelstraße**; STREETNO: 5 **9**; ZIP: 91386 **52118**; CITY: Oberbaumgarten **Kotzenbüll**; PHONE: 312/4603663 **644/1281306**; URL: http://oqq.yyzlnom.zbm/hiog-z30270.ubgo **http://gdv.doxulye.doz/fwqj-g78597.nqju**


SAMPLE INPUT: 3
---------------
Das ist das Problem von HKV und Rossner überhaupt.

--=20
Zitat:
ACHTUNG "Turbotrottel"
Jeuststraße 8, 85283 Baldern, 278/9147652
http://yci.lovvwoz.tvw/zkau-v03379.uhmv

SAMPLE OUTPUT: 3
----------------
ORG: HKV **Triagon**; ORG: Rossner **Arzum**; STREET: Jeuststraße **Dreijochgasse**; STREETNO: 8 **2**; ZIP: 85283 **38524**; CITY: Baldern **Ahmsen**; PHONE: 278/9147652 **001/0373780**; URL: http://yci.lovvwoz.tvw/zkau-v03379.uhmv **http://ruj.vftcqyi.cyb/jovm-t59381.pfuk**


SAMPLE INPUT: 4
---------------
* Ilka Ullenboom <Dtmvh.Zneiwl@h-jsrfcx.wg>:

[matrix]

Du magst auch MiB nicht, oder?

Henning
-- 
cross veinless

SAMPLE OUTPUT: 4
----------------
FEMALE: Ilka **Carole**; FAMILY: Ullenboom **Ulferts**; EMAIL: Dtmvh.Zneiwl@h-jsrfcx.wg **Zdcxc.Axdfyh@u-cuebhp.we**; MALE: Henning **Valerian**


SAMPLE INPUT: 5
---------------
Findet ihr unter:

http://bgl.dwkmuvqugt-hfmtsqaj.zd/Dsmpgv


Schaut mal rein
Rjaffc

SAMPLE OUTPUT: 5
----------------
URL: http://bgl.dwkmuvqugt-hfmtsqaj.zd/Dsmpgv **http://jmr.bquhhzahku-xsfnqcua.ry/Scgjyp**; USER: Rjaffc **Xlaczq**


SAMPLE INPUT: 6
---------------
und 

pennymarkt v.94 o.O.
WPK 138076

Prinzipiell interessieren mich aber auch andere Genuss-Scheine

Stefan

SAMPLE OUTPUT: 6
----------------
ORG: pennymarkt **Zeitungen&Zeitschriften**; UFID: 138076 **GKE 330952**; MALE: Stefan **Ulfert**


SAMPLE INPUT: 7
---------------
On Fri, 12. 02. 22 19:54:25 +0100, Anton Hauptmanns

Ja wer macht denn sowas ?

-- 
Artur Lüdeck
jlcrjl@dqyjm.gf
http://bre.gedzrmlsq.qc/
Mobile: 0656-5242408

SAMPLE OUTPUT: 7
----------------
DATE: 12. 02. 22 **03. 06. 20**; MALE: Anton **Otmar**; FAMILY: Hauptmanns **Olte**; MALE: Artur **Oswald**; FAMILY: Lüdeck **Freischläger**; EMAIL: jlcrjl@dqyjm.gf **lzvjme@nylof.of**; URL: http://bre.gedzrmlsq.qc/ **http://ojf.oxewmmrcr.kq/**; PHONE: 0656-5242408 **0028-3487683**



INPUT
-----
The following is the text for which you will provide output:

'''

In [5]:
input_text = '''
Jo, wieviele Web-Buchungssysteme zeigen vollständige 
Tarifbedingungen an? Und wieviele Reisebüros betrachten
Tarifregeln als "nicht für Dritte"? Dass sie nicht unbedingt
allgemeinverständlich formuliert sind, kommt erschwerend
hinzu.

Gruß,
Kai

-- 
Kai Marklstorfer                           howm@ic.wk.rqe-aq.aa
http://usz.hwjp.vem-nq.wz/~wdhp/        zbip@yjyg.cyw-pj.io
mobile: +59 096 2819 361
'''

payload = {
    "model": model,
    "messages": [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": input_text
        }
    ],
    "stream": False
}


response = requests.post(url, json=payload).json()
print(response.get('message', {}).get('content', response.get('error')))

MALE: Kai **Siegfried**; FAMILY: Marklstorfer **Kleinsteuber**; EMAIL: howm@ic.wk.rqe-aq.aa **cswzg@fmxkm.gxq-qaz.ni**; URL: http://usz.hwjp.vem-nq.wz/~wdhp/ **http://hjv.hfuzrxbw.pn-rf.yu/jqzg-j98217.tpg**; PHONE: +59 096 2819 361 **+33 0156 7412 198**
